<a href="https://colab.research.google.com/github/lmcanavals/topics/blob/main/0602_fixture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ortools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 48.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 12.1 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3


In [15]:
nombre_federacion = ['UEFA', 'AFC', "CAF", "Conmebol", "Concacaf", "Repechaje"]
nombre_equipo = ['Bélgica', 'Francia', 'Inglaterra', 'España', 'Portugal', 
                 'Países Bajos', 'Dinamarca', 'Alemania', 'Suiza', 'Croacia', 
                 'Serbia', 'Polonia', 
                 'UEFA1', 
                 'Catar', 
                 'Irán', 'Japón', 'Corea del Sur', 
                 'Arabia Saudita', 
                 'Senegal', 'Marruecos', 'Túnez', 'Camerún', 'Ghana',
                 'Brasil', 'Argentina', 
                 'Uruguay',
                 'Ecuador', 
                 'México', 'EEUU', 
                 'Canadá',
                 'AFC/Conmebol', 'OFC/Concacaf']

In [13]:
from ortools.sat.python import cp_model

model = cp_model.CpModel()

grupos = []
for i in range(len(nombre_equipo)):
    grupos.append([model.NewBoolVar(f"grupo_{i}_{j}") for j in range(8)])
print(grupos)

federacion = [model.NewIntVar(1, 6, f"federacion_{i}") for i in range(32)]
print(federacion)

bombo = [model.NewIntVar(1, 4, f"bombo_{i}") for i in range(32)]
print(bombo)

# 1    2   3   4         5           6
# UEFA AFC CAF Conmebol  Concacaf	Repechaje
# 13   5   5   4         3           2
for i in range(32):
    if i < 13:
        model.Add(federacion[i] == 1)
    elif i < 18:
        model.Add(federacion[i] == 2)
    elif i < 23:
        model.Add(federacion[i] == 3)
    elif i < 27:
        model.Add(federacion[i] == 4)
    elif i < 30:
        model.Add(federacion[i] == 5)
    else:
        model.Add(federacion[i] == 6)

for i in range(32):
    if i in [0, 1, 2, 3, 4] + [13] + [23, 24]:
        model.Add(bombo[i] == 1)
    elif i in [5, 6, 7, 8, 9] + [25] + [27, 28]:
        model.Add(bombo[i] == 2)
    elif i in [10, 11] + [14, 15, 16] + [18, 19, 20]:
        model.Add(bombo[i] == 3)
    else:
        model.Add(bombo[i] == 4)

for i in range(32):
    model.Add(sum(grupos[i]) == 1)

for j in range(8):
    model.Add(sum([grupos[i][j] for i in range(32)]) == 4)

grupos_federacion = []
grupos_bombo = []
for i in range(32):
    grupos_federacion.append([model.NewIntVar(0, 1000, f"gf_{i}_{j}") for j in range(8)])
    grupos_bombo.append([model.NewIntVar(0, 1000, f"gv_{i}_{j}") for j in range(8)])
print(grupos_federacion)
print(grupos_bombo)

for i in range(32):
    for j in range(8):
        model.AddMultiplicationEquality(grupos_federacion[i][j], [federacion[i], grupos[i][j]])
        model.AddMultiplicationEquality(grupos_bombo[i][j], [bombo[i], grupos[i][j]])

def count_vars(model, x, val, c):
    b = [model.NewBoolVar(f"b{i}") for i in range(len(x))]
    for i in range(len(x)):
        model.Add(x[i] == val).OnlyEnforceIf(b[i])
        model.Add(x[i] != val).OnlyEnforceIf(b[i].Not())
    model.Add(c == sum(b))

count = {}
for j in range(8):
    for k in range(1,7):
        count[(k, j)] = model.NewIntVar(0, 100, f"count_{k}_{j}")
        count_vars(model, [grupos_federacion[i][j] for i in range(32)], k, count[(k, j)])
        if k == 1:
            model.Add(count[(k, j)] <= 2)
        elif k > 1:
            model.Add(count[(k, j)] <= 1)


count_bombos = {}
for j in range(8):
    for k in range(1,5):
        count_bombos[(k, j)] = model.NewIntVar(0, 100, f"cb_{k}_{j}")
        count_vars(model, [grupos_bombo[i][j] for i in range(32)], k, count_bombos[(k, j)])
        model.Add(count_bombos[(k, j)] <= 1)

[[grupo_0_0(0..1), grupo_0_1(0..1), grupo_0_2(0..1), grupo_0_3(0..1), grupo_0_4(0..1), grupo_0_5(0..1), grupo_0_6(0..1), grupo_0_7(0..1)], [grupo_1_0(0..1), grupo_1_1(0..1), grupo_1_2(0..1), grupo_1_3(0..1), grupo_1_4(0..1), grupo_1_5(0..1), grupo_1_6(0..1), grupo_1_7(0..1)], [grupo_2_0(0..1), grupo_2_1(0..1), grupo_2_2(0..1), grupo_2_3(0..1), grupo_2_4(0..1), grupo_2_5(0..1), grupo_2_6(0..1), grupo_2_7(0..1)], [grupo_3_0(0..1), grupo_3_1(0..1), grupo_3_2(0..1), grupo_3_3(0..1), grupo_3_4(0..1), grupo_3_5(0..1), grupo_3_6(0..1), grupo_3_7(0..1)], [grupo_4_0(0..1), grupo_4_1(0..1), grupo_4_2(0..1), grupo_4_3(0..1), grupo_4_4(0..1), grupo_4_5(0..1), grupo_4_6(0..1), grupo_4_7(0..1)], [grupo_5_0(0..1), grupo_5_1(0..1), grupo_5_2(0..1), grupo_5_3(0..1), grupo_5_4(0..1), grupo_5_5(0..1), grupo_5_6(0..1), grupo_5_7(0..1)], [grupo_6_0(0..1), grupo_6_1(0..1), grupo_6_2(0..1), grupo_6_3(0..1), grupo_6_4(0..1), grupo_6_5(0..1), grupo_6_6(0..1), grupo_6_7(0..1)], [grupo_7_0(0..1), grupo_7_1(0..1)

In [16]:
solver = cp_model.CpSolver()
status = solver.Solve(model)
if status == cp_model.OPTIMAL:
    print("\t", end="")
    for i in range(32):
        print(solver.Value(federacion[i]), end="\t")
    print()

    print("\t", end="")
    for i in range(32):
        print(i + 1, end="\t")
    print()

    for j in range(8):
        print(f"G{j+1}", end="->\t")
        for i in range(32):
            print(solver.Value(grupos[i][j]), end="\t")
        print()
    print()

    for j in range(8):
        print(f"G{j+1}", end="->\t")
        for i in range(32):
            print(solver.Value(grupos_bombo[i][j]), end="\t")
        print()
    print()

    for j in range(8):
        print(f"G{j+1}", end="->\t")
        for i in range(32):
            if solver.Value(grupos[i][j]):
                s = f"{nombre_equipo[i]}({nombre_federacion[solver.Value(federacion[i])-1]})"
                print(f"{s:20}", end="\t")
        [print(solver.Value(count[(k, j)]), end=" ") for k in range(1, 7)]
        print(end="\t")
        [print(solver.Value(count_bombos[(k, j)]), end=" ") for k in range(1, 5)]
        print()


	1	1	1	1	1	1	1	1	1	1	1	1	1	2	2	2	2	2	3	3	3	3	3	4	4	4	4	5	5	5	6	6	
	1	2	3	4	5	6	7	8	9	10	11	12	13	14	15	16	17	18	19	20	21	22	23	24	25	26	27	28	29	30	31	32	
G1->	0	0	1	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	1	0	0	0	0	0	0	0	0	1	0	0	1	
G2->	1	0	0	0	0	0	0	0	1	0	0	0	0	0	1	0	0	0	0	0	0	0	1	0	0	0	0	0	0	0	0	0	
G3->	0	0	0	0	0	0	1	0	0	0	1	0	0	0	0	0	0	1	0	0	0	0	0	0	1	0	0	0	0	0	0	0	
G4->	0	1	0	0	0	1	0	0	0	0	0	0	0	0	0	0	0	0	0	0	1	0	0	0	0	0	0	0	0	1	0	0	
G5->	0	0	0	0	0	0	0	1	0	0	0	1	0	1	0	0	0	0	0	0	0	1	0	0	0	0	0	0	0	0	0	0	
G6->	0	0	0	1	0	0	0	0	0	0	0	0	0	0	0	0	0	0	1	0	0	0	0	0	0	0	1	1	0	0	0	0	
G7->	0	0	0	0	0	0	0	0	0	1	0	0	0	0	0	0	1	0	0	0	0	0	0	1	0	0	0	0	0	0	1	0	
G8->	0	0	0	0	1	0	0	0	0	0	0	0	1	0	0	1	0	0	0	0	0	0	0	0	0	1	0	0	0	0	0	0	

G1->	0	0	1	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	3	0	0	0	0	0	0	0	0	2	0	0	4	
G2->	1	0	0	0	0	0	0	0	2	0	0	0	0	0	3	0	0	0	0	0	0	0	4	0	0	0	0	0	0	0	0	0	
G3->	0	0	0	0	0	0	2	0	0	0	3	0	0	0	0	0	0	4	0	0	0	0	0	0	1	0	0	0	0	0	0	0	
G4->	0	1	0	0	0	2	0	0	0	0	0	0	0	0	0	0	0	0	0	0	3	0	0	0	0	0	0	0	0	4	0	0	
G5->

In [ ]:
class VarArraySolutionPrinter(cp_model.CpSolverSolutionCallback):
  def __init__(self, variables):
    cp_model.CpSolverSolutionCallback.__init__(self)
    self.__variables = variables
    self.__solution_count = 0

  def on_solution_callback(self):
    self.__solution_count += 1
    for j in range(8):
      print('G'+str(j+1),end='->\t')
      for i in range(32):
        if self.Value(self.__variables[i][j]):
          s = nombre_equipo[i]
          print(s+' '*(20-len(s)),end='\t')
      print()
    print()

  def solution_count(self):
    return self.__solution_count

In [ ]:
solver = cp_model.CpSolver()
solution_printer = VarArraySolutionPrinter(grupos)
# Enumerate all solutions.
solver.parameters.enumerate_all_solutions = True
# Solve.
status = solver.Solve(model, solution_printer)